# Third Round of Full Day RFI Flagging Using 2D-Filtered SNRs


**by Josh Dillon**, last updated March 27, 2025

This notebook brings together the results of [single-baseline 2D DPSS filtering notebook](https://github.com/HERA-Team/hera_notebook_templates/blob/master/notebooks/single_baseline_2D_filtered_SNRs.ipynb) to make a set of flagging decisions prior to inpainting. This approach is iterative, and very similar to [Round 2 flagging](https://github.com/HERA-Team/hera_notebook_templates/blob/master/notebooks/full_day_rfi_round_2.ipynb), though it includes special treatment of TV allocations ([see HERA Memo #82 for more details](https://reionization.org/wp-content/uploads/2013/03/HERA082_TV_Info.pdf)). 

Here's a set of links to skip to particular figures and tables:
# [• Figure 1: Waterfall of Maximum z-Score of Either Polarization Before Round 3 Flagging](#Figure-1:-Waterfall-of-Maximum-z-Score-of-Either-Polarization-Before-Round-3-Flagging)
# [• Figure 2: Histogram of z-Scores](#Figure-2:-Histogram-of-z-Scores)
# [• Figure 3: Waterfall of Maximum z-Score of Either Polarization After Round 3 Flagging](#Figure-3:-Waterfall-of-Maximum-z-Score-of-Either-Polarization-After-Round-3-Flagging)
# [• Figure 4: Summary of Flags Before and After Round 3 Flagging](#Figure-4:-Summary-of-Flags-Before-and-After-Round-3-Flagging)

In [ ]:
import time
tstart = time.time()
!hostname

In [ ]:
import os
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'
import h5py
import hdf5plugin  # REQUIRED to have the compression plugins available
import numpy as np
import yaml
import glob
import re
import matplotlib
from scipy.signal import convolve, convolve2d
from pyuvdata import UVFlag
from hera_qm import xrfi
from hera_cal import io, flag_utils
from hera_filters import dspec
import matplotlib.pyplot as plt

from IPython.display import display, HTML
%matplotlib inline
display(HTML("<style>.container { width:100% !important; }</style>"))
_ = np.seterr(all='ignore')  # get rid of red warnings
%config InlineBackend.figure_format = 'retina'

In [ ]:
RED_AVG_FILE = os.environ.get("RED_AVG_FILE", None)
# RED_AVG_FILE = '/lustre/aoc/projects/hera/jsdillon/H6C/IDR3/2459861/zen.2459861.25364.sum.smooth_calibrated.red_avg.uvh5'

CORNER_TURN_MAP_YAML = os.environ.get("CORNER_TURN_MAP_YAML", 
                                        os.path.join(os.path.dirname(RED_AVG_FILE), "single_baseline_files/corner_turn_map.yaml"))
SNR_SUFFIX = os.environ.get("SNR_SUFFIX", ".2Dfilt_SNR.uvh5")
OUTFILE = os.environ.get("OUTFILE", None)
if OUTFILE is None:
    jdstr = [s for s in os.path.basename(RED_AVG_FILE).split('.') if s.isnumeric()][0]
    OUTFILE = os.path.basename(RED_AVG_FILE).split(jdstr)[0] + jdstr + '.flag_waterfall_round_3.h5'
    OUTFILE = os.path.join(os.path.dirname(CORNER_TURN_MAP_YAML), OUTFILE)

MIN_SAMP_FRAC = float(os.environ.get("MIN_SAMP_FRAC", .15))
FM_LOW_FREQ = float(os.environ.get("FM_LOW_FREQ", 87.5)) # in MHz
FM_HIGH_FREQ = float(os.environ.get("FM_HIGH_FREQ", 108.0)) # in MHz

Z_THRESH = float(os.environ.get("Z_THRESH", 4))
WS_Z_THRESH = float(os.environ.get("WS_Z_THRESH", 2))
AVG_Z_THRESH = float(os.environ.get("AVG_Z_THRESH", 1))
MAX_FREQ_FLAG_FRAC = float(os.environ.get("MAX_FREQ_FLAG_FRAC", .25))
MAX_TIME_FLAG_FRAC = float(os.environ.get("MAX_TIME_FLAG_FRAC", .25))

TV_CHAN_EDGES = os.environ.get("TV_CHAN_EDGES", "174,182,190,198,206,214,222,230,238,246,254")

FREQ_CONV_SIZE  = float(os.environ.get("FREQ_CONV_SIZE", 8.0)) # in MHz

for setting in ['RED_AVG_FILE', 'CORNER_TURN_MAP_YAML', 'SNR_SUFFIX', 'TV_CHAN_EDGES', 'OUTFILE']:
    print(f'{setting} = "{eval(setting)}"')
for setting in ['MIN_SAMP_FRAC', 'FM_LOW_FREQ', 'FM_HIGH_FREQ', 'Z_THRESH', 'WS_Z_THRESH',
                'AVG_Z_THRESH', 'MAX_FREQ_FLAG_FRAC', 'MAX_TIME_FLAG_FRAC', 'FREQ_CONV_SIZE']:
    print(f'{setting} = {eval(setting)}')

In [ ]:
with open(CORNER_TURN_MAP_YAML, 'r') as file:
    corner_turn_map = yaml.unsafe_load(file)

In [ ]:
all_snr_files = [snr_file.replace('.uvh5', SNR_SUFFIX) 
                 for snr_files in corner_turn_map['files_to_outfiles_map'].values() 
                 for snr_file in snr_files]
extant_snr_files = [snr_file for snr_file in all_snr_files if os.path.exists(snr_file)]
print(f'Found {len(extant_snr_files)} SNR files, starting with {extant_snr_files[0]}')

In [ ]:
# get autocorrelations
all_outfiles = [outfile for outfiles in corner_turn_map['files_to_outfiles_map'].values() for outfile in outfiles]
for outfile in all_outfiles:
    match = re.search(r'\.(\d+)_(\d+)\.', os.path.basename(outfile))
    if match and match.group(1) == match.group(2):
        hd_autos = io.HERAData(outfile)
        _, _, auto_nsamples = hd_autos.read(polarizations=['ee', 'nn'])
        break

med_auto_nsamples = {pol: np.median(auto_nsamples[0,0,pol]) for pol in ['ee', 'nn']}

In [ ]:
# define slices for TV allocations
tv_edges = [float(edge) for edge in TV_CHAN_EDGES.split(',')]
tv_slices = []
for i in range(len(tv_edges) - 1):
    chans_in_band = np.argwhere((hd_autos.freqs / 1e6 > tv_edges[i]) & (hd_autos.freqs / 1e6 < tv_edges[i+1]))
    if len(chans_in_band) > 0:
        tv_slices.append(slice(np.min(chans_in_band), np.max(chans_in_band) + 1))

In [ ]:
# load up SNRs, counts, and nsamples
hd = io.HERADataFastReader(extant_snr_files[0])
abs_SNR_sums = {}
abs_SNR_counts = {}
abs_SNR_med_nsamples = {}

# for snr_file in tqdm(extant_snr_files[0:5]):
for snr_file in extant_snr_files:
    hd = io.HERADataFastReader(snr_file)
    data, flags, nsamples = hd.read()
    for bl in data:
        abs_SNR_sums[bl] = np.where(flags[bl], 0, np.abs(data[bl]))
        abs_SNR_counts[bl] = np.where(flags[bl], 0, 1)
        abs_SNR_med_nsamples[bl] = np.median(nsamples[bl] * ~flags[bl])

In [ ]:
# combine SNRs incoherently, excluding those with too few samples
abs_SNR_sum = {pol: np.zeros((len(hd.times), len(hd.freqs)), dtype=float) for pol in hd.pols}
abs_SNR_count = {pol: np.zeros((len(hd.times), len(hd.freqs)), dtype=float) for pol in hd.pols}
bls_used = []
for bl in abs_SNR_sums:
    if np.median(abs_SNR_med_nsamples[bl]) > MIN_SAMP_FRAC * med_auto_nsamples[bl[2]]:
        if np.linalg.norm(hd.antpos[bl[0]] - hd.antpos[bl[1]]) > 1:
            bls_used.append(bl)
            abs_SNR_sum[bl[2]] += abs_SNR_sums[bl]
            abs_SNR_count[bl[2]] += abs_SNR_counts[bl]

In [ ]:
# convert SNRs to a z-score
zscore = {}
for pol in abs_SNR_sum.keys():
    predicted_mean = 1.0
    sigma = predicted_mean * np.sqrt(2 / np.pi)
    variance_expected = (4 - np.pi) / 2 * sigma**2 / abs_SNR_count[pol]
    zscore[pol] = (abs_SNR_sum[pol] / abs_SNR_count[pol] - predicted_mean) / variance_expected**.5
    zscore[pol] = np.where(abs_SNR_count[pol] == 0, np.nan, zscore[pol])

In [ ]:
# recenter z-scores above and below FM and per-polarization
_, (low_band, high_band) = flag_utils.get_minimal_slices(np.any(~np.isfinite(list(zscore.values())), axis=0), 
                                                         freqs=data.freqs, freq_cuts=[FM_LOW_FREQ / 2 + FM_HIGH_FREQ / 2])
for pol in zscore:
    for band in [low_band, high_band]:
        zscore[pol][:, band] -= np.nanmedian(zscore[pol][:, band])    

## Plotting Functions

In [ ]:
def plot_max_z_score(zscore, flags=None, vmin=-5, vmax=5):
    if flags is None:
        flags = np.any(~np.isfinite(list(zscore.values())), axis=0)
    plt.figure(figsize=(14,10), dpi=300)
    extent = [data.freqs[0] / 1e6, data.freqs[-1] / 1e6, 
              data.times[-1] - int(data.times[0]), data.times[0] - int(data.times[0])]
    
    plt.imshow(np.where(flags, np.nan, np.nanmax([zscore['ee'], zscore['nn']], axis=0)), aspect='auto', 
               cmap='coolwarm', interpolation='none', vmin=vmin, vmax=vmax, extent=extent)
    plt.colorbar(location='top', label='Max z-score of either polarization', extend='both', aspect=40, pad=.02)
    plt.xlabel('Frequency (MHz)')
    plt.ylabel(f'JD - {int(data.times[0])}')
    plt.tight_layout()
    for freq in tv_edges:
        if freq < data.freqs[-1] * 1e-6:
            plt.axvline(freq, lw=.5, ls='--', color='k')

In [ ]:
def plot_histogram():
    plt.figure(figsize=(14,4), dpi=100)
    bins = np.arange(-50, 100, .1)
    hist_ee = plt.hist(np.ravel(zscore['ee']), bins=bins, density=True, label='ee-polarized z-scores', alpha=.5)
    hist_nn = plt.hist(np.ravel(zscore['nn']), bins=bins, density=True, label='nn-polarized z-scores', alpha=.5)
    plt.plot(bins, (2*np.pi)**-.5 * np.exp(-bins**2 / 2), 'k:', label='Gaussian approximate\nnoise-only distribution')
    plt.axvline(WS_Z_THRESH, c='r', ls='--', label='Watershed z-score')
    plt.axvline(Z_THRESH, c='r', ls='-', label='Threshold z-score')    
    plt.yscale('log')
    all_densities = np.concatenate([hist_ee[0][hist_ee[0] > 0], hist_nn[0][hist_nn[0] > 0]]) 
    plt.ylim(np.min(all_densities) / 2, np.max(all_densities) * 2)
    plt.xlim([-50, 100])
    plt.legend()
    plt.xlabel('z-score')
    plt.ylabel('Density')
    plt.tight_layout()

In [ ]:
def summarize_flagging(flags):
    plt.figure(figsize=(14,10), dpi=200)
    cmap = matplotlib.colors.ListedColormap(((0, 0, 0),) + matplotlib.cm.get_cmap("Set2").colors[0:2])
    extent = [data.freqs[0] / 1e6, data.freqs[-1] / 1e6, 
              data.times[-1] - int(data.times[0]), data.times[0] - int(data.times[0])]    
    plt.imshow(np.where(np.any(~np.isfinite(list(zscore.values())), axis=0), 1, np.where(flags, 2, 0)), 
               aspect='auto', cmap=cmap, interpolation='none', extent=extent)
    plt.clim([-.5, 2.5])
    cbar = plt.colorbar(location='top', aspect=40, pad=.02)
    cbar.set_ticks([0, 1, 2])
    cbar.set_ticklabels(['Unflagged', 'Flagged After Round 2', 'Flagged After Round 3'])
    plt.xlabel('Frequency (MHz)')
    plt.ylabel(f'JD - {int(data.times[0])}')
    plt.tight_layout()

# Figure 1: Waterfall of Maximum z-Score of Either Polarization Before Round 3 Flagging

This figure shows the worse (higher z-score) of the two polarizations. Dotted lines in the high band show TV allocations, which recieve special treatment. Large positive excursions are problem and likely need flagging. note that below and FM are handled separately and may have different levels of post-flag filtering.

In [ ]:
plot_max_z_score(zscore)

# Figure 2: Histogram of z-Scores

Shows a comparison of the histogram of z-scores to a Gaussian approximation of what one might expect from thermal noise. Without filtering, the actual distribution is a weighted sum of Rayleigh distributions. Filtering further complicates this. To make the z-scores more reliable, a single per-polarization and per-band median is subtracted from each waterfall, which allows us to flag low-level outliers with more confidence. Any points beyond the solid red line are flagged. Any points neighboring a flag beyond the dashed red line are also flagged. Finally, flagging is performed for low-level outliers on whole times or channels, in TV allocations, and other compact regions in frequency.



In [ ]:
plot_histogram()

## Flagging functions

In [ ]:
def iteratively_flag_on_averaged_zscore(flags, zscore, avg_func=np.nanmean, avg_z_thresh=AVG_Z_THRESH, verbose=True):
    '''Flag whole integrations or channels based on average z-score. This is done
    iteratively to prevent bad times affecting channel averages or vice versa.'''

    _, (low_band, high_band) = flag_utils.get_minimal_slices(flags, freqs=data.freqs, freq_cuts=[(FM_LOW_FREQ + FM_HIGH_FREQ) * .5e6])
    flagged_chan_count = 0
    flagged_int_count = {low_band: 0, high_band: 0}
    for band in (low_band, high_band):
        while True:
            zspec = avg_func(np.where(flags, np.nan, zscore)[:, band], axis=0)
            ztseries = avg_func(np.where(flags, np.nan, zscore)[:, band], axis=1)
    
            if (np.nanmax(zspec) < avg_z_thresh) and (np.nanmax(ztseries) < avg_z_thresh):
                break
    
            if np.nanmax(zspec) >= np.nanmax(ztseries):
                flagged_chan_count += np.sum((zspec >= np.nanmax(ztseries)) & (zspec >= avg_z_thresh))
                flags[:, band][:, (zspec >= np.nanmax(ztseries)) & (zspec >= thresh)] = True
            else:
                flagged_int_count[band] += np.sum((ztseries >= np.nanmax(zspec)) & (ztseries >= avg_z_thresh))
                flags[(ztseries >= np.nanmax(zspec)) & (ztseries >= avg_z_thresh), band] = True

    ztseries_low = avg_func(np.where(flags, np.nan, zscore)[:, low_band], axis=1)
    flags[(ztseries_low > avg_z_thresh) & np.all(flags[:, high_band], axis=1), low_band] = True
    
    if verbose:
        if (flagged_int_count[low_band] > 0) or (flagged_int_count[high_band] > 0) or (flagged_chan_count > 0):
            print(f'\tFlagging an additional {flagged_int_count[low_band]} low-band integrations, '
                  f'{flagged_int_count[high_band]} high-band integrations, and {flagged_chan_count} channels.')

def impose_max_chan_flag_frac(flags, max_flag_frac=MAX_FREQ_FLAG_FRAC, verbose=True):
    '''Flag channels already flagged more than max_flag_frac (excluding completely flagged times).'''
    _, (low_band, high_band) = flag_utils.get_minimal_slices(flags, freqs=data.freqs, freq_cuts=[(FM_LOW_FREQ + FM_HIGH_FREQ) * .5e6])
    for band in [low_band, high_band]:
        unflagged_times = ~np.all(flags[:, band], axis=1)
        frequently_flagged_chans =  np.mean(flags[unflagged_times, band], axis=0) >= max_flag_frac
        if verbose:
            flag_diff_count = np.sum(frequently_flagged_chans) - np.sum(np.all(flags[:, band], axis=0))
            if flag_diff_count > 0:
                print(f'\tFlagging {flag_diff_count} channels previously flagged {max_flag_frac:.2%} or more.')        
        flags[:, band][:, frequently_flagged_chans] = True
        
def impose_max_time_flag_frac(flags, max_flag_frac=MAX_TIME_FLAG_FRAC, verbose=True):
    '''Flag times already flagged more than max_flag_frac (excluding completely flagged channels).'''
    _, (low_band, high_band) = flag_utils.get_minimal_slices(flags, freqs=data.freqs, freq_cuts=[(FM_LOW_FREQ + FM_HIGH_FREQ) * .5e6])
    for name, band in zip(['low', 'high'], [low_band, high_band]):
        unflagged_chans = ~np.all(flags[:, band], axis=0)
        frequently_flagged_times =  np.mean(flags[:, band][:, unflagged_chans], axis=1) >= max_flag_frac
        if verbose:
            flag_diff_count = np.sum(frequently_flagged_times) - np.sum(np.all(flags[:, band], axis=1))
            if flag_diff_count > 0:
                print(f'\tFlagging {flag_diff_count} {name}-band times previously flagged {max_flag_frac:.2%} or more.')
        flags[frequently_flagged_times, band] = True

def flag_tv(flags, zscore, tv_thresh=AVG_Z_THRESH, egregious_thresh=(2 * Z_THRESH)):
    '''Flag single-time TV allocations with average zscores above tv_thresh, excluding particularly bad channels.'''
    for pol in zscore:
        for tvs in tv_slices:
            for tind in range(zscore[pol].shape[0]):
                if np.nanmean(zscore[pol][tind, tvs]) > tv_thresh:
                    egregious_outliers = zscore[pol][tind, tvs] > egregious_thresh
                    if np.nanmean(zscore[pol][tind, tvs][~egregious_outliers]) > tv_thresh:
                        # even without the worst outliers, it still looks like there's TV in this whole allocation
                        flags[tind, tvs] = True
                    else:
                        # just flag the most egregious outliers
                        flags[tind, tvs] |= egregious_outliers

def watershed_flag(flags, zscore, ws_z_thresh=WS_Z_THRESH):
    '''Wrapper around xrfi._ws_flag_waterfall to be performed separately above and below FM.'''
    while True:        
        nflags = np.sum(flags)
        _, (low_band, high_band) = flag_utils.get_minimal_slices(flags, freqs=data.freqs, freq_cuts=[(FM_LOW_FREQ + FM_HIGH_FREQ) * .5e6])
        for band in [low_band, high_band]:
            for pol in ['ee', 'nn']:
                flags[:, band] |= xrfi._ws_flag_waterfall(zscore[pol][:, band], flags[:, band], ws_z_thresh)
        if np.sum(flags) == nflags:
            break

def iterative_freq_conv_flagging(flags, zscore, conv_size=FREQ_CONV_SIZE, one_chan_thresh=Z_THRESH, full_kernel_thresh=AVG_Z_THRESH):
    '''Looks for streteches of increasing size that fit a decreasing threshold. At conv_size (in MHz), it flags 
    stretches with average z-score above full_kernel_thresh. At one pixel, it uses one_chan_thresh.
    In between, it interpolates logarithmically.'''
    _, (low_band, high_band) = flag_utils.get_minimal_slices(flags, freqs=data.freqs, freq_cuts=[(FM_LOW_FREQ + FM_HIGH_FREQ) * .5e6])
    df_MHz = np.median(np.diff(data.freqs)) / 1e6
    widths = np.array([int(w) + 1 for w in 2**np.arange(1, np.ceil(np.log2(FREQ_CONV_SIZE / df_MHz) + np.finfo(float).eps))])
    
    # prevent any widths from being so big that they mix high and low bands
    max_width = (high_band.start - low_band.stop) * 2
    widths[widths > max_width] = max_width
    widths = np.unique(widths)

    # Create cuts that get more strict as the kernel gets bigger
    cuts = one_chan_thresh * (full_kernel_thresh / one_chan_thresh)**((widths - 1) / (conv_size / df_MHz - 1))

    for width, cut in zip(widths, cuts):
        result = {}
        for pol in zscore.keys():
            kernel = np.ones((1, int(width)), dtype=float)
            mask = ~(np.isnan(zscore[pol]) | flags)
            filled_data = np.where(mask, zscore[pol], 0.0)
            conv_data = convolve2d(filled_data, kernel, mode='same')
            conv_mask = convolve2d(mask.astype(float), kernel, mode='same')
            with np.errstate(divide='ignore', invalid='ignore'):
                result[pol] = conv_data / conv_mask

        for band in [low_band, high_band]:
            above_cut = np.any([result[pol][:, band] > cut for pol in result.keys()], axis=0)
            flags[:, band] |= (convolve2d(above_cut.astype(float), kernel, mode='same') > 0)
        
        print(f'{np.mean(flags):.3%} of waterfall flagged after {width}-channel convolution-based flagging with z-scores above {cut:.3f}.')

## Main Flagging Routine

In [ ]:
flags = np.any(~np.isfinite(list(zscore.values())), axis=0)
_, (low_band, high_band) = flag_utils.get_minimal_slices(flags, freqs=data.freqs, freq_cuts=[100e6])
print(f'{np.mean(flags):.3%} of waterfall flagged to start.')

# flag bad TV allocations
flag_tv(flags, zscore)
print(f'{np.mean(flags):.3%} of waterfall flagged after TV channel cuts.')

# flag whole integrations or channels using outliers in median
while True:
    nflags = np.sum(flags)
    for pol in ['ee', 'nn']:    
        iteratively_flag_on_averaged_zscore(flags, zscore[pol], avg_func=np.nanmedian, avg_z_thresh=AVG_Z_THRESH, verbose=True)
        impose_max_chan_flag_frac(flags, max_flag_frac=MAX_FREQ_FLAG_FRAC, verbose=True)
        impose_max_time_flag_frac(flags, max_flag_frac=MAX_TIME_FLAG_FRAC, verbose=True)
    if np.sum(flags) == nflags:
        break  
print(f'{np.mean(flags):.3%} of waterfall flagged after flagging whole times and channels with median z > {AVG_Z_THRESH}.')

# flag largest outliers
_, (low_band, high_band) = flag_utils.get_minimal_slices(flags, freqs=data.freqs, freq_cuts=[(FM_LOW_FREQ + FM_HIGH_FREQ) * .5e6])
for band in [low_band, high_band]:
    for pol in ['ee', 'nn']:
        flags[:, band] |= (zscore[pol][:, band] > Z_THRESH) 
print(f'{np.mean(flags):.3%} of waterfall flagged after flagging z > {Z_THRESH} outliers.')

# watershed flagging
watershed_flag(flags, zscore, ws_z_thresh=WS_Z_THRESH)
print(f'{np.mean(flags):.3%} of waterfall flagged after watershed flagging on z > {WS_Z_THRESH} neighbors of prior flags.')
iterative_freq_conv_flagging(flags, zscore, conv_size=FREQ_CONV_SIZE, one_chan_thresh=Z_THRESH, full_kernel_thresh=AVG_Z_THRESH)
print(f'{np.mean(flags):.3%} of waterfall flagged after channel convolution flagging.')

# watershed flagging
watershed_flag(flags, zscore, ws_z_thresh=WS_Z_THRESH)
print(f'{np.mean(flags):.3%} of waterfall flagged after watershed flagging again on z > {WS_Z_THRESH} neighbors of prior flags.')

# flag whole integrations or channels using outliers in mean
while True:
    nflags = np.sum(flags)
    for pol in ['ee', 'nn']:    
        iteratively_flag_on_averaged_zscore(flags, zscore[pol], avg_func=np.nanmean, avg_z_thresh=AVG_Z_THRESH, verbose=True)
        impose_max_chan_flag_frac(flags, max_flag_frac=MAX_FREQ_FLAG_FRAC, verbose=True)
        impose_max_time_flag_frac(flags, max_flag_frac=MAX_TIME_FLAG_FRAC, verbose=True)
    if np.sum(flags) == nflags:
        break  
print(f'{np.mean(flags):.3%} of waterfall flagged after flagging whole times and channels with average z > {AVG_Z_THRESH}.')

# watershed flagging again
watershed_flag(flags, zscore, ws_z_thresh=WS_Z_THRESH)
print(f'{np.mean(flags):.3%} of waterfall flagged after watershed flagging one last time on z > {WS_Z_THRESH} neighbors of prior flags.')

# Figure 3: Waterfall of Maximum z-Score of Either Polarization After Round 3 Flagging

Same as [Figure 1](#Figure-1:-Waterfall-of-Maximum-z-Score-of-Either-Polarization-Before-Round-3-Flagging) above, but now with additional flagging from this round.

In [ ]:
plot_max_z_score(zscore, flags=flags)

# Figure 4: Summary of Flags Before and After Round 3 Flagging

This plot shows which times and frequencies were flagged before and after this notebook. It is directly comparable to Figure 5 of the first round [full_day_rfi](https://github.com/HERA-Team/hera_notebook_templates/blob/master/notebooks/full_day_rfi.ipynb) notebook, as well as Figure 5 of the [full_day_rfi_round_2](https://github.com/HERA-Team/hera_notebook_templates/blob/master/notebooks/full_day_rfi_round_2.ipynb) notebook.





In [ ]:
summarize_flagging(flags)

## Save results

In [ ]:
uvf = UVFlag(hd_autos, mode='flag', waterfall=True)
for polind in range(uvf.flag_array.shape[2]):
    uvf.flag_array[:, :, polind] = flags

uvf.write(OUTFILE, clobber=True)

## Metadata

In [ ]:
for repo in ['hera_cal', 'hera_qm', 'hera_filters', 'hera_notebook_templates', 'pyuvdata']:
    exec(f'from {repo} import __version__')
    print(f'{repo}: {__version__}')

In [ ]:
print(f'Finished execution in {(time.time() - tstart) / 60:.2f} minutes.')